# **Housing Prices Competition for Kaggle Learn Users**

___

## Competition Description.

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

## Evaluation

### Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

### Metric
Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. (Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.)

### Submission File Format
The file should contain a header and have the following format:

Id,SalePrice
<br>
1461,169000.1
<br>
1462,187724.1233
<br>
1463,175221
<br>
etc.

There is a sample file called 'sample_submission' in 'Base de Dados' folder.

___

## **Importando as bibliotecas necessárias:**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

## **Lendo o arquivo de entrada:**

In [ ]:
caminho_arquivo = '../Base de Dados/train.csv'

pd.set_option('display.max_columns', None)

house_price_data = pd.read_csv(caminho_arquivo, index_col=0)
house_price_data.head()

## **Análise exploratória dos dados:**

## Observações preliminares

In [ ]:
house_price_data.tail()

In [ ]:
house_price_data.shape

O "shape" do dataset está mostrando que ele possui 1460 linhas e 80 colunas, sendo que a coluna [SalePrice] é o target que o modelo deve prever.
Ou seja, tem 79 colunas que podem ser usadas para a seleção dos recursos.

**Colunas Númericas**

In [ ]:
# Lista das colunas númericas
colunas_numericas = house_price_data.select_dtypes(exclude=['object'])
colunas_numericas.columns

In [ ]:
# Quantidade de colunas númericas
len(colunas_numericas.columns)

Acima mostra que, aparentemente, tem 37 colunas do tipo númerico, incluindo a target [SalePrice].

- Algumas anomalias nas colunas do dataset podem fazer com que o tipo de dado da coluna se torne 'object', isso pode ocasionar em uma separação errada das colunas númericas e categoricas.
- Também é possível que tenham colunas que possuem dados em uma forma mais discreta e números com valores limitados. Essas colunas devem ser interpretadas, também, como categoricas.

As 37 colunas numéricas possuem as seguintes caracteristicas gerais:

In [ ]:
colunas_numericas.describe().round(decimals=2)

**Colunas Categoricas**

In [ ]:
colunas_categoricas = house_price_data.select_dtypes(include=['object'])
colunas_categoricas.columns

In [ ]:
len(colunas_categoricas.columns)

Existem 43 colunas categoricas com as seguintes caracteristicas gerais:

In [ ]:
colunas_categoricas.describe()

## Explorando as colunas númericas

**Distorção da coluna target**

Aparentemente é uma boa prática minimizar a distorção do dataset. A razão é que, geralmente, os dados distorcidos afetam negativamente a precisão da previsão dos modelos de regressão.
Nota: Embora seja importante para a regressão linear, corrigir a distorção não é necessária para a Árvore de Decisão ou Florestas Aleatórias.

In [ ]:
target = house_price_data.SalePrice

plt.figure()

sns.distplot(target)

plt.title('Distribuição da SalePrice')
plt.show()

In [ ]:
sns.distplot(np.log(target))

plt.title('Distribuição do Log-transformado SalePrice')
plt.xlabel('log(SalePrice)')
plt.show()

In [ ]:
print('SalePrice possui uma distorção de ' + str(target.skew().round(decimals=2)) + ' enquando o log-transformado SalePrice melhora a distorção para ' + str(np.log(target).skew().round(decimals=2)))

**Distribuição das colunas**

In [ ]:
colunas_numericas = house_price_data.select_dtypes(exclude='object').drop('SalePrice', axis=1).copy()

fig = plt.figure(figsize=(12,18))
for i in range(len(colunas_numericas.columns)):
    fig.add_subplot(9,4,i+1)
    sns.distplot(colunas_numericas.iloc[:,i].dropna())
    plt.xlabel(colunas_numericas.columns[i])

plt.tight_layout()
plt.show()

Também é possível fazer um log transformado nas seguinte distorções:

LotFrontage, LotArea, stFlrSF, GrLivArea, OpenPorchSF

**Buscando Outliers**

Análise univariada - box plots para atributos numéricos

In [ ]:
fig = plt.figure(figsize=(12, 18))

for i in range(len(colunas_numericas.columns)):
    fig.add_subplot(9, 4, i + 1)
    
    sns.boxplot(y=colunas_numericas.iloc[:, i])
    
plt.tight_layout()
plt.show()

Análise bivariada - gráficos de dispersão para atributos alvo versus atributos numéricos

In [ ]:
f = plt.figure(figsize=(12,20))

for i in range(len(colunas_numericas.columns)):
    f.add_subplot(9, 4, i+1)
    sns.scatterplot(x=colunas_numericas.iloc[:,i], y=target)
    
plt.tight_layout()
plt.show()

Em uma primeira vista do scatter plots contra o SalePrice, parecer ter outliers nas seguintes colunas:

- LotFrontage
- LotArea
- BsmtFinSF1
- TotalBsmtSF
- 1stFlrSF
- GrLivArea
- LowQualFinSF

**Correlações entre as colunas**

In [ ]:
correlacao = house_price_data.corr(numeric_only=True)

f, asx = plt.subplots(figsize=(14, 12))

plt.title('Correlação das colunas numericas', size=16)

sns.heatmap(correlacao)
# sns.heatmap(correlacao, annot=True)

plt.show()

Usando como referência o target SalePrice, as top 15 correlações são:

In [ ]:
correlacao['SalePrice'].sort_values(ascending=False).head(15)

**Valores faltantes ou nulos nas colunas númericas**

In [ ]:
# Mostrar a coluna com a maior quantidade de nulos
colunas_numericas.isna().sum().sort_values(ascending=False).head()

## Explorando as colunas categoricas

In [ ]:
colunas_categoricas = house_price_data.select_dtypes(include='object').columns

print(colunas_categoricas)

In [ ]:
var = house_price_data['KitchenQual']

f, ax = plt.subplots(figsize=(10, 6))

sns.boxplot(y=house_price_data.SalePrice, x=var)

plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))

sns.boxplot(y=house_price_data.SalePrice, x=house_price_data.Neighborhood)

plt.xticks(rotation=40)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12.5, 4))

sns.countplot(x='Neighborhood', data=house_price_data)

plt.xticks(rotation=90)
plt.ylabel('Frequency')
plt.show()

**Valores faltantes ou nulos nas colunas categoricas**

In [ ]:
house_price_data[colunas_categoricas].isna().sum().sort_values(ascending=False).head(17)

## **Limpeza dos dados e pré-processamento**

## Lidando com valores faltantes ou nulos

In [ ]:
# Criando uma cópia do dataset
house_price_data_copy = house_price_data.copy()

house_price_data_copy.MasVnrArea = house_price_data_copy.MasVnrArea.fillna(0)
house_price_data_copy.LotFrontage = house_price_data_copy.LotFrontage.fillna(0)
house_price_data_copy.GarageYrBlt = house_price_data_copy.GarageYrBlt.fillna(0)

colunas_categoricas_preenche_none = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                     'GarageCond', 'GarageQual', 'GarageFinish', 'GarageType',
                     'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond',
                     'MasVnrType', 'Electrical']

for categoria in colunas_categoricas_preenche_none:
    house_price_data_copy[categoria] = house_price_data_copy[categoria].fillna('None')
    
house_price_data_copy.isna().sum().sort_values(ascending=False).head()

In [ ]:
house_price_data_copy.head()

## Lidando com os outliers

In [ ]:
# Removendo os outliers de acordo com as observações do scatter plot contra a coluna SalePrice
house_price_data_copy = house_price_data_copy.drop(house_price_data_copy['LotFrontage'][house_price_data_copy['LotFrontage']>200].index)
house_price_data_copy = house_price_data_copy.drop(house_price_data_copy['LotArea'][house_price_data_copy['LotArea']>100000].index)
house_price_data_copy = house_price_data_copy.drop(house_price_data_copy['BsmtFinSF1'][house_price_data_copy['BsmtFinSF1']>4000].index)
house_price_data_copy = house_price_data_copy.drop(house_price_data_copy['TotalBsmtSF'][house_price_data_copy['TotalBsmtSF']>6000].index)
house_price_data_copy = house_price_data_copy.drop(house_price_data_copy['1stFlrSF'][house_price_data_copy['1stFlrSF']>4000].index)
house_price_data_copy = house_price_data_copy.drop(house_price_data_copy['GrLivArea'][(house_price_data_copy['GrLivArea']>4000) & (target<300000)].index)
house_price_data_copy = house_price_data_copy.drop(house_price_data_copy['LowQualFinSF'][house_price_data_copy['LowQualFinSF']>550].index)

In [ ]:
house_price_data_copy.head()

## Transformando o dado para reduzir a distorção.

No momento só vamos utilizar a váriavel target [SalePrice]

In [ ]:
house_price_data_copy['SalePrice'] = np.log(house_price_data_copy['SalePrice'])
house_price_data_copy = house_price_data_copy.rename(columns={'SalePrice': 'SalePrice_Log'})

Validando como ficou o dataset

In [ ]:
house_price_data_copy.head()

## **Feature Selection & Engineering**

**Considerando correlações altas**

Utilizando correlações altas para um algoritmo de machine learning pode causar a redução na performance.

In [ ]:
correlacao_transformada = house_price_data_copy.corr(numeric_only=True)

plt.figure(figsize=(12, 10))

sns.heatmap(correlacao_transformada)

Colunas com alta correlação entre si (a coluna da esquerda possui uma correlação maior com o SalePrice).

- GarageCars and GarageArea (0.882)
- YearBuild and GarageYrBlt (0.826)
- GrLivArea and TotRmsAbvGrd (0.826)
- TotalBsmtSF and '1stFlrSG' (0.780)

Provavelmente devemos escolher, entre as que possuem mais de 0.8 na correlação, se fazemos um drop nas colunas que possuem a menor correlação contra a coluna target [SalePrice].

In [ ]:
colunas_desejadas = ['GarageCars', 'GarageArea', 'YearBuilt', 'GarageYrBlt', 'GrLivArea', 'TotRmsAbvGrd', 'TotalBsmtSF', '1stFlrSF']

correlacao_transformada['SalePrice_Log'][colunas_desejadas].sort_values(ascending=False)

**Realizando o feature scaling, e a transformação das colunas categoricas.**

In [ ]:
# Removendo as colunas identificadas no scatter plot e na correlação
colunas_drop = ['SalePrice_Log', 'MiscVal', 'MSSubClass', 'MoSold', 'YrSold', 'GarageArea', 'GarageYrBlt', 'TotRmsAbvGrd']

X = house_price_data_copy.drop(colunas_drop, axis=1)
y = house_price_data_copy.SalePrice_Log

# OneHotEnconding para transformar as colunas categoricas
X = pd.get_dummies(X)

# Separando os dados em treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalonamento e imputação nos dados de treinamento
simple_imputer = SimpleImputer(strategy='constant', fill_value=-1)
standard_scaler = StandardScaler()

X_treino = standard_scaler.fit_transform(X_treino)
X_treino = simple_imputer.fit_transform(X_treino)

# Escalonamento e imputação nos dados de teste (sem ajuste)
X_teste = standard_scaler.transform(X_teste)
X_teste = simple_imputer.transform(X_teste)

In [ ]:
X_treino.shape

In [ ]:
X_teste.shape

## **Avaliação preliminar de algoritmos de Machine Learning**

In [ ]:
def inverte_y(y_transformado):
    return np.exp(y_transformado)

In [ ]:
y_teste = inverte_y(y_teste)
y_teste

In [ ]:
# Series to collate mean absolute errors for each algorithm
mae_compare = pd.Series()
mae_compare.index.name = 'Algorithm'

# Specify Model ================================

# Random Forest. =============================
rf_model = RandomForestRegressor(random_state=5)
rf_model.fit(X_treino, y_treino)
rf_val_predictions = rf_model.predict(X_teste)
rf_val_predictions = inverte_y(rf_val_predictions)
rf_val_mae = mean_absolute_error(rf_val_predictions, y_teste)

mae_compare['RandomForest'] = rf_val_mae

# XGBoost. Define the model. ======================================
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
xgb_model.fit(X_treino, y_treino, early_stopping_rounds=5, 
              eval_set=[(X_teste, y_teste)], verbose=False)
xgb_val_predictions = xgb_model.predict(X_teste)
xgb_val_predictions = inverte_y(xgb_val_predictions)
xgb_val_mae = mean_absolute_error(xgb_val_predictions, y_teste)

mae_compare['XGBoost'] = xgb_val_mae

# Árvore de decisão
dt_model = DecisionTreeRegressor(max_depth=4)
dt_model.fit(X_treino, y_treino)
dt_val_predictions = dt_model.predict(X_teste)
dt_val_predictions = inverte_y(dt_val_predictions)
dt_val_mae = mean_absolute_error(dt_val_predictions, y_teste)

mae_compare['DecisionTree'] = dt_val_mae

# Linear Regression =================================================
# Para esse modelo o Linear Regression não funciona se for utilizado o StandardScaler
#linear_model = LinearRegression()
#linear_model.fit(X_treino, y_treino)
#linear_val_predictions = linear_model.predict(X_teste)
#linear_val_predictions = inverte_y(linear_val_predictions)
#linear_val_mae = mean_absolute_error(linear_val_predictions, y_teste)

#mae_compare['LinearRegression'] = linear_val_mae
# print("Validation MAE for Linear Regression Model: {:,.0f}".format(linear_val_mae))

# Lasso ==============================================================
lasso_model = Lasso(alpha=0.0005, random_state=5)
lasso_model.fit(X_treino, y_treino)
lasso_val_predictions = lasso_model.predict(X_teste)
lasso_val_predictions = inverte_y(lasso_val_predictions)
lasso_val_mae = mean_absolute_error(lasso_val_predictions, y_teste)

mae_compare['Lasso'] = lasso_val_mae

# Ridge ===============================================================
ridge_model = Ridge(alpha=0.002, random_state=5)
ridge_model.fit(X_treino, y_treino)
ridge_val_predictions = ridge_model.predict(X_teste)
ridge_val_predictions = inverte_y(ridge_val_predictions)
ridge_val_mae = mean_absolute_error(ridge_val_predictions, y_teste)

mae_compare['Ridge'] = ridge_val_mae

# ElasticNet ===========================================================
elastic_net_model = ElasticNet(alpha=0.02, random_state=5, l1_ratio=0.7)
elastic_net_model.fit(X_treino, y_treino)
elastic_net_val_predictions = elastic_net_model.predict(X_teste)
elastic_net_val_predictions = inverte_y(elastic_net_val_predictions)
elastic_net_val_mae = mean_absolute_error(elastic_net_val_predictions, y_teste)

mae_compare['ElasticNet'] = elastic_net_val_mae

# Gradient Boosting Regression ==========================================
gbr_model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, 
                                      max_depth=4, random_state=5)
gbr_model.fit(X_treino, y_treino)
gbr_val_predictions = gbr_model.predict(X_teste)
gbr_val_predictions = inverte_y(gbr_val_predictions)
gbr_val_mae = mean_absolute_error(gbr_val_predictions, y_teste)

mae_compare['GradientBoosting'] = gbr_val_mae

print('MAE values for different algorithms:')
mae_compare.sort_values(ascending=True).round()

In [ ]:
# Criando o gráfico de barras
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(mae_compare) + 1), mae_compare.values, color='red', linestyle='dashed', marker='o', markerfacecolor='blue', markersize=10)
plt.xticks(range(1, len(mae_compare) + 1), mae_compare.index)
plt.title('Avaliação Preliminar de Modelos de Machine Learning')
plt.xlabel('Algoritmo')
plt.ylabel('Erro Médio Quadrático')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

**Validação cruzada**

In [ ]:
imputer = SimpleImputer()
imputed_X = imputer.fit_transform(X)
n_folds = 10

In [ ]:
scores = cross_val_score(lasso_model, imputed_X, y, scoring='neg_mean_squared_error', cv=n_folds)
lasso_mae_scores = np.sqrt(-scores)

print('For LASSO model:')
print('Mean RMSE = ' + str(lasso_mae_scores.mean().round(decimals=3)))
print('Error std deviation = ' + str(lasso_mae_scores.std().round(decimals=3)))

In [ ]:
scores = cross_val_score(gbr_model, imputed_X, y, scoring='neg_mean_squared_error', cv=n_folds)
gbr_mae_scores = np.sqrt(-scores)

print('For Gradient Boosting model:')
print('Mean RMSE = ' + str(gbr_mae_scores.mean().round(decimals=3)))
print('Error std deviation = ' + str(gbr_mae_scores.std().round(decimals=3)))

In [ ]:
scores = cross_val_score(xgb_model, imputed_X, y, scoring='neg_mean_squared_error', cv=n_folds)
xgb_mae_scores = np.sqrt(-scores)

print('For XGBoost model:')
print('Mean RMSE = ' + str(xgb_mae_scores.mean().round(decimals=3)))
print('Error std deviation = ' + str(xgb_mae_scores.std().round(decimals=3)))

In [ ]:
scores = cross_val_score(rf_model, imputed_X, y, scoring='neg_mean_squared_error', cv=n_folds)
rf_mae_scores = np.sqrt(-scores)

print('For Random Forest model:')
print('Mean RMSE = ' + str(rf_mae_scores.mean().round(decimals=3)))
print('Error std deviation = ' + str(rf_mae_scores.std().round(decimals=3)))

In [ ]:
scores = cross_val_score(dt_model, imputed_X, y, scoring='neg_mean_squared_error', cv=n_folds)
dt_mae_scores = np.sqrt(-scores)

print('For Decision Tree model:')
print('Mean RMSE = ' + str(dt_mae_scores.mean().round(decimals=3)))
print('Error std deviation = ' + str(dt_mae_scores.std().round(decimals=3)))

## **Seleção do melhor algoritmo e ajustes finos**

**Criando um modelo e fazendo as predições**

In [ ]:
# Validando os melhores parâmetros do modelo Lasso
param_grid = [{'alpha': [0.0007, 0.0005, 0.005]}]

top_reg = Lasso()

grid_search = GridSearchCV(top_reg, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(imputed_X, y)

grid_search.best_params_

In [ ]:
# Caminho do arquivo que será utilizado para as predições.
test_data_path = '../Base de Dados/test.csv'

test_data = pd.read_csv(test_data_path)
test_data.head()

**Repetindo o pré-processamento definido anteriormente**

In [ ]:
X_teste_predicao = test_data.copy()
X_teste_predicao.head()

In [ ]:
X_teste_predicao.shape

In [ ]:
# Colunas númericas
X_teste_predicao.MasVnrArea = X_teste_predicao.MasVnrArea.fillna(0)

# Retirando as colunas escolhidas
for categoria in colunas_categoricas_preenche_none:
    X_teste_predicao[categoria] = X_teste_predicao[categoria].fillna('None')
    
if 'SalePrice_Log' in colunas_drop:
    colunas_drop.remove('SalePrice_Log')
    
X_teste_predicao = X_teste_predicao.drop(colunas_drop, axis=1)    

# OneHotEncoder
X_teste_predicao = pd.get_dummies(X_teste_predicao)

# Tendo certeza que os dados de teste estão encodados da mesma forma que os dados de treino
treino_final, teste_final = X.align(X_teste_predicao, join='left', axis=1)

teste_final_imputed = simple_imputer.transform(teste_final)

teste_final_imputed.shape

**Criando o modelo final**

In [ ]:
# Utilizando o melhor modelo: Lasso
modelo_final = Lasso(alpha=0.0005, random_state=42)

treino_final_imputed = simple_imputer.fit_transform(treino_final)

modelo_final.fit(treino_final_imputed, y)

**Realizando a previsão**

In [ ]:
predicao_teste = modelo_final.predict(teste_final_imputed)

output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': np.exp(predicao_teste)})

output.to_csv('..\Base de Dados\submission.csv', index=False)